In [1]:
from collections import OrderedDict

import numpy as np
import trimesh
import open3d as o3d
import torch
import os
import glob


from src.polar_traversal import gather_all_paths, sample_all_paths

np.set_printoptions(precision=4, suppress=True)
torch.set_printoptions(precision=4, sci_mode=False)

In [2]:
stl_dir = "./data/face_stl/"
stls = [os.path.join(stl_dir, f) 
        for f in os.listdir(stl_dir) 
        if f.endswith(".stl")]

stls.sort()
# Problems 26, 45, 69
idx = 12
file_name = stls[idx]

file_name = './data/centered_3.stl'
print(idx, file_name)

s_no =  512
latitudes_num = s_no * 2
paths = gather_all_paths(file_name, latitudes_num)
print(len(paths))

samples_num = s_no // 2
samples = sample_all_paths(paths, samples_num)
vertices, normals = samples
vertices.shape, normals.shape

12 ./data/centered_3.stl
1024


((512, 512, 3), (512, 512, 3))

In [ ]:
points = vertices.reshape(-1, 3)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)    
o3d.visualization.draw_geometries([pcd])

In [ ]:
torch.cat((torch.tensor(vertices), 
             torch.tensor(normals)), dim=-1).shape

In [ ]:
blueprint = OrderedDict({
    'vertices': torch.tensor(vertices).permute(2, 0, 1)[None].float(),
    'normals': torch.tensor(normals).permute(2, 0, 1)[None].float(),
})
torch.save(blueprint, './data/blueprint.pt')

In [ ]:
blueprint = torch.load('./data/blueprint.pt')
blueprint['vertices'].shape, blueprint['normals'].shape

In [ ]:
from src.renderer import Renderer
from src.config import get_parser

device =  torch.device("cuda")

parser  = get_parser()
opt = parser.parse_args(args=[])
opt

renderer = Renderer(opt).to(device)
renderer

In [ ]:
from pytorch3d.ops import sample_points_from_meshes


from pytorch3d.structures import (
    Pointclouds,
    Meshes,
)
from pytorch3d.io import load_obj
from pytorch3d.renderer import (
    FoVOrthographicCameras, 
    FoVPerspectiveCameras, look_at_view_transform, look_at_rotation,
    RasterizationSettings, MeshRenderer, MeshRasterizer, BlendParams,
    SoftSilhouetteShader, HardPhongShader, PointLights, TexturesVertex,
)

# Set the cuda device
device = torch.device("cuda:0")
torch.cuda.set_device(device)

# Load the obj and ignore the textures and materials.
verts, faces_idx, _ =  load_obj("./data/teapot.obj")
faces = faces_idx.verts_idx

# Initialize each vertex to be white in color.
verts_rgb = torch.ones_like(verts)[None] # (1, V, 3)
textures =  TexturesVertex(verts_features=verts_rgb.to(device))

# Create a Meshes object for the teapot. Here we have only one mesh in the batch
teapot_mesh = Meshes(
    verts=[verts.to(device)],
    faces=[faces.to(device)],
    textures=textures,
)
teapot_mesh

n_points = 1000000
vertices, normals = sample_points_from_meshes(teapot_mesh, 
    n_points, return_normals=True)

vertices =  vertices[0].to(device)
normals =  normals[0].to(device)

rgb = torch.ones((n_points, 4), device=device) - 0.3

point_cloud = Pointclouds(points=[vertices], 
                          normals=[normals],
                          features=[rgb],)


import matplotlib.pyplot as plt

images =  renderer(point_cloud)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.grid("off")
plt.axis("off");

In [ ]:
from src.util import make_faces

blueprint = torch.load('./data/blueprint.pt')
blueprint['vertices'].shape, blueprint['normals'].shape

In [ ]:
w, h, _ = blueprint['vertices'].shape
faces = make_faces(w, h)
faces.shape

In [ ]:
from pytorch3d.structures import Meshes

verts = blueprint['vertices'].reshape(-1, 3)
faces_idx = torch.tensor(faces)
mesh = Meshes(verts=[verts], faces=[faces_idx])
mesh

In [ ]:
norms = mesh.faces_normals_list()[0]
norms.shape

In [ ]:
blueprint['normals'].reshape(-1, 3).shape

In [ ]:
verts_normals = mesh.verts_normals_list()[0]
verts_normals.shape

In [ ]:
norms = mesh.faces_normals_list()[0]
norms.shape

In [ ]:
import matplotlib.pyplot as plt

images =  renderer(pt_cloud)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.grid("off")
plt.axis("off");

In [ ]:
vertices.dtype,normals, rgb

In [ ]:
mesh_1 = trimesh.Trimesh(vertices=verts_p.cpu(), faces=faces)
mesh_1.show()

In [ ]:
file = stls[idx].split('/')[-1].replace('.stl', '.npz')
np.savez_compressed('./data/512/'+file, samples=samples)

In [ ]:
latitudes_num = 1024
samples_num = 256

for idx, stl in enumerate(stls):
    print(idx, stl)
    file_name = stls[idx]
    print(idx, file_name)    
    paths = gather_all_paths(file_name, latitudes_num)
    samples = sample_all_paths(paths, samples_num)
    file = file_name.split('/')[-1].replace('.stl', '.npz')
    np.savez_compressed('./data/512/'+file, samples=samples)

In [ ]:
len(paths[100])